In [1]:
import pandas as pd
import numpy as np


In [2]:
!wget https://raw.githubusercontent.com/dherman/wc-demo/master/data/shakespeare-plays.csv


--2025-07-09 17:49:09--  https://raw.githubusercontent.com/dherman/wc-demo/master/data/shakespeare-plays.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6124558 (5.8M) [text/plain]
Saving to: ‘shakespeare-plays.csv’

shakespeare-plays.c 100%[===================>]   5.84M  --.-KB/s    in 0.06s   

2025-07-09 17:49:10 (95.5 MB/s) - ‘shakespeare-plays.csv’ saved [6124558/6124558]



In [3]:
df = pd.read_csv("shakespeare-plays.csv", on_bad_lines="skip")
df.columns = ['Title', 'ActSceneLine', 'Character', 'Line']
df.head()

,Title,ActSceneLine,Character,Line
0,I,I,BERTRAM,"And I in going, madam, weep o'er my father's d..."
1,I,I,BERTRAM,"anew: but I must attend his majesty's command, to"
2,I,I,BERTRAM,"whom I am now in ward, evermore in subjection."
3,I,I,LAFEU,"You shall find of the king a husband, madam; you,"
4,I,I,LAFEU,"sir, a father: he that so generally is at all ..."


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106216 entries, 0 to 106215
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   Title         106216 non-null  object
 1   ActSceneLine  106216 non-null  object
 2   Character     106210 non-null  object
 3   Line          106216 non-null  object
dtypes: object(4)
memory usage: 3.2+ MB


In [5]:
df.isnull().sum()


,0
Title,0
ActSceneLine,0
Character,6
Line,0


In [6]:
df.duplicated().sum()

np.int64(136)

In [7]:
df.value_counts()

Title  ActSceneLine  Character          Line                                        
V      I             Clown              With hey, ho, &c.                               4
IV     I             DESDEMONA          My lord?                                        4
II     III           Porter             Knock,                                          4
V      II            EMILIA             My husband!                                     3
       I             LORENZO            In such a night                                 3
                                                                                       ..
II     III           HENRY BOLINGBROKE  Stands for my bounty. But who comes here?       1
                                        Shall be your love and labour's recompense.     1
                                        Pluck'd from my arms perforce and given away    1
                                        On what condition stands it and wherein?        1
                                        Which I have sworn to weed and pluck away.      1
Name: count, Length: 106074, dtype: int64

In [8]:
df=df.dropna()

In [9]:
texts = df['Line'].dropna().astype(str).tolist()
corpus = " ".join(texts).lower()
print("Sample:", corpus[:500])


Sample: and i in going, madam, weep o'er my father's death anew: but i must attend his majesty's command, to whom i am now in ward, evermore in subjection. you shall find of the king a husband, madam; you, sir, a father: he that so generally is at all times good must of necessity hold his virtue to you; whose worthiness would stir it up where it wanted rather than lack it where there is such abundance. what hope is there of his majesty's amendment? he hath abandoned his physicians, madam; under whose pr


In [10]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch




In [11]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [12]:
inputs = tokenizer(corpus, return_tensors="pt", max_length=1024, truncation=True)
inputs.input_ids.shape


torch.Size([1, 1024])

In [16]:
from torch.optim import AdamW

model.train()
optimizer = AdamW(model.parameters(), lr=5e-5)

for epoch in range(3):
    outputs = model(**inputs, labels=inputs["input_ids"])
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    print(f"Epoch {epoch + 1} Loss: {loss.item():.4f}")


Epoch 1 Loss: 4.1123
Epoch 2 Loss: 3.9374
Epoch 3 Loss: 3.7622


In [17]:
def generate_text(prompt, max_length=50):
    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    with torch.no_grad():
        outputs = model.generate(
            input_ids,
            max_length=max_length,
            num_return_sequences=1,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=0.9,
            no_repeat_ngram_size=2,
            pad_token_id=tokenizer.eos_token_id
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [18]:
import gradio as gr

demo = gr.Interface(
    fn=generate_text,
    inputs="text",
    outputs="text",
    title="Shakespeare Generator",
    description="Enter a prompt and the AI will complete the next words like Shakespeare!"
)

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8f86a5fa4517f12c86.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
